<h1> DattoRMM - Servicability Report </h1>

# Import Modules and Define Variables

In [ ]:
# data import and file manipulation
import os
import requests
import json
import csv
import xlrd

#data conditioning
import pandas as pd
import numpy as np
import re
import datetime as dt

In [ ]:
 # add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# dictionary Directory
dictionary_dir = 'd:/git/example_infrastructure_data_dev/dictionaries'

# source dir for nable exported data
source_folder = 'd:/project_docs/abc_nable_migration/abc_nable_exports/patch_management'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports'

### Import Modules, Create env Variables

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

In [ ]:
# import and assign secrets from env.ini

dattormm_config = config['dattormm']

base_uri = dattormm_config['base_uri']

## Create auth token

In [ ]:
# call token api url
token_uri = f'{base_uri}/auth/oauth/token'


# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = dattormm_config['api_key']
data['password'] = dattormm_config['api_secret']

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']

## Create DataFrame via API Call Iteration


In [ ]:

## Create Devices DataFrame
# request content response
request_url = f'{base_uri}/api/v2/account/devices'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


# iterate and combine remaining pages
df_devices = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)

    df_current_page = pd.DataFrame(c_dict['devices'])
    df_devices = pd.concat([df_devices, df_current_page], ignore_index=False)

In [ ]:
c_dict['devices']

# Data Shaping

## Create New Columns from Dictionary Columns

### Type | Category

In [ ]:
def device_category(device):
    if device == None:
        return None
    else:
        return device['category']

In [ ]:
def device_type(device):
    if device == None:
        return None
    else:
        return device['type']

In [ ]:
df_devices['category'] = df_devices['deviceType'].apply(device_category)
df_devices['type'] = df_devices['deviceType'].apply(device_type)

# Rename 'type' values to split devices into (2) : 'computer' or 'server'
#df_devices['type'].replace({'Desktop':'computer','Laptop':'computer','Server':'server'},inplace=True)

In [ ]:
df_devices.drop(columns='deviceType',inplace=True)

### Patch Managment Breakdown
 patchStatus | patchesApprovedPending | patchesNotApproved | patchesInstalled

In [ ]:
# patchStatus
def patch_status(patch_managment):
    return patch_managment['patchStatus']

df_devices['patchStatus'] = df_devices['patchManagement'].apply(patch_status)

# patchesApprovedPending
def patches_approved_pending(patch_management):
    return patch_management['patchesApprovedPending']

df_devices['patchesApprovedPending'] = df_devices['patchManagement'].apply(patches_approved_pending)

# patchesNotApproved
def patches_not_approved(patch_managment):
    return patch_managment['patchesNotApproved']

df_devices['patchesNotApproved'] = df_devices['patchManagement'].apply(patches_not_approved)

# patchesInstalled
def patches_installed(patch_management):
    return patch_management['patchesInstalled']

df_devices['patchesInstalled'] = df_devices['patchManagement'].apply(patches_installed)


# drop patchManagement {inplace=True}
df_devices.drop('patchManagement',axis=1,inplace=True)

### Patch Percent Patched Calculated Column
(Compliance Percentage = PatchesApproved / Patches Installed)

## Drop 'antivirus'

In [ ]:
df_devices.drop('antivirus',axis=1,inplace=True)

## Create Time Columns and Timedate Shaping

### Add Timezone Column from UDF

In [ ]:
# Timezone
def local_timezone(udf):
    return udf['udf10']

df_devices['localTimezone'] = df_devices['udf'].apply(local_timezone)

# drop udf {inplace=True}
df_devices.drop('udf',axis=1,inplace=True)

### Create Date Correlation Columns

In [ ]:
# all date columns
parse_dates =  ['lastAuditDate','lastSeen','lastReboot','creationDate',]

### Convert Epoch to UTC

In [ ]:
df_devices['lastAuditDate'] = pd.to_datetime(df_devices['lastAuditDate'],unit='ms',errors='coerce')
#df_devices['lastAuditDate'].head(5)

In [ ]:
df_devices['lastSeen'] = pd.to_datetime(df_devices['lastSeen'],unit='ms',errors='coerce')
#df_devices['lastSeen'].head(5)

In [ ]:
df_devices['creationDate'] = pd.to_datetime(df_devices['creationDate'],unit='ms',errors='coerce')
#df_devices['creationDate'].head(5)

In [ ]:
df_devices['lastReboot'] = pd.to_datetime(df_devices['lastReboot'],unit='ms',errors='coerce')
#df_devices['lastReboot'].head(5)

### Define and apply functions to create correlation columns

In [ ]:
def no_audit_7_days(last_audit):
    if last_audit < dt.datetime.now() - dt.timedelta(days=7):
        return 1
    else:
        return 0

In [ ]:
def no_audit_30_days(last_audit):
    if last_audit < dt.datetime.now() - dt.timedelta(days=30):
        return 1
    else:
        return 0

In [ ]:
def offline_30_days(last_seen):
    if last_seen < dt.datetime.now() - dt.timedelta(days=30):
        return 1
    else:
        return 0

In [ ]:
def no_reboot_30_days(last_reboot):
    if last_reboot < dt.datetime.now() - dt.timedelta(days=30):
        return 1
    else:
        return 0

In [ ]:
# Create Column - Devices Last Audit > 7 days
df_devices['noAudit7Days'] = df_devices['lastAuditDate'].apply(no_audit_7_days)

In [ ]:
# Create Column - Devices Last Audit > 7 days
df_devices['noAudit30Days'] = df_devices['lastAuditDate'].apply(no_audit_30_days)

In [ ]:
# Create Column - Devices Offline 30 Days
df_devices['offline30Days'] = df_devices['lastSeen'].apply(offline_30_days)

In [ ]:
# Create Column - Last Reboot Extended Duration and Online without Reboot Extended Duration
df_devices['noReboot30Days'] = df_devices['lastReboot'].apply(no_reboot_30_days)

In [ ]:
# Create DF copy for reference
df_raw_data = df_devices

In [ ]:
df_devices['patchStatusPercent'] = round( 100 - ((df_devices['patchesApprovedPending'] / ((df_devices['patchesApprovedPending'] + df_devices['patchesInstalled']) ) * 100)),2)

## Device OS EOL isEOL and daysTillEol

### EOL OS List as Parse

In [ ]:
df_product_release_info = pd.read_csv(f'{git_folder}/dictionaries/product_release_info.dict')
ms_eol_os = []
for index,row in df_product_release_info.iterrows():
    eol_dict = {}
    osname = row['product']
    oseol = row['releaseEndDate']
    eol_dict[osname] = oseol
    ms_eol_os.append(eol_dict)

In [ ]:
def addEOL(operatingSystem):
    for os in ms_eol_os:
        for k,v in os.items():
            if operatingSystem == k:
                return v

In [ ]:
def isEOL(osEndofLifeDate):
    try:
        timedelta = dt.datetime.now() - osEndofLifeDate
        if timedelta.days > 0:
            return 1
        else:
            return 0
    except Exception as e:
        print(e)

In [ ]:
df_devices['osEndofLifeDate'] = df_devices['operatingSystem'].apply(addEOL)

In [ ]:
df_release_info = pd.read_csv(f'{dictionary_dir}/product_release_info.dict')

In [ ]:
def match_release(row):
    try:
        result = re.match(r'^(Windows (XP|7|8|8.1|10|11|Server))([\s\d\w]+)?',row['product'])
        print(row['product'])
        print('-'*50)
        print(result.group(1))
        # print(result.group(2))
        print(result.group(3).lstrip())
        print(row['edition'])
        if row['release'] != "[NaN]":
            print(row['release'])
        print('*'*50)
    except:
        pass

In [ ]:
windows_mask = df_release_info['product'].str.contains('^Windows (XP|7|8|8.1|10|11|Server|Server\s?\d+?\s?\d+?)')
for index, row in df_release_info[windows_mask].fillna('[NaN]').iterrows():
    match_release(row)

In [ ]:
df_devices

In [ ]:
aaaa

In [ ]:
df_devices_timefix = pd.to_datetime(df_devices['osEndofLifeDate'],unit='ns')
df_devices['osEndofLifeDate'] = df_devices_timefix.values.astype('datetime64[s]')
#df_devices.drop('osEndofLifeDate',axis=1,inplace=True)

In [ ]:
df_devices['isEOL'] = df_devices['osEndofLifeDate'].apply(isEOL)

## DattoRMM DataFrame Data Standardization Shaping

### Hostname to_upper()

In [ ]:
df_devices['hostname'] = df_devices['hostname'].str.upper()

### Replace Dtypes with Int64

In [ ]:
convert_to_int_mask = ((df_devices.dtypes == 'float') | (df_devices.dtypes == 'bool') | (df_devices.dtypes == 'uint8')) & (df_devices.columns != 'patchStatusPercent')
convert_to_int = df_devices.dtypes[convert_to_int_mask].index.tolist()

In [ ]:
df_devices[convert_to_int] = df_devices[convert_to_int].astype('Int64')

In [ ]:
df_devices

## Add Dummy Columns

### Add 'patchStatus' Dummy Columns

In [ ]:
df_patch_status = pd.get_dummies(df_devices['patchStatus'],prefix='patchStatus',prefix_sep='')
df_patch_status.drop('patchStatusNoPolicy',axis=1, inplace=True)
df_devices = df_devices.join(df_patch_status)
df_devices.drop('patchStatus',axis=1,inplace=True)

### Add 'category' Dummy Columns

In [ ]:
df_category = pd.get_dummies(df_devices['category'],prefix='is',prefix_sep='')
df_category.drop('isDesktop',axis=1,inplace=True)
df_devices = df_devices.join(df_category)
df_devices.drop('category',axis=1,inplace=True)

## Add Report Creation Date Info Column

In [ ]:
report_creation_date = (dt.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))
df_devices['reportCreationDate'] = report_creation_date

# Create Hardware Audit DataFrame

In [ ]:
df_devices

In [ ]:
def hardware_api_req(row):
    # request content response
    request_url = f"{dattormm_config['base_uri']}/api/v2/audit/device/{row['uid']}"

    # construct header
    headers = CaseInsensitiveDict()
    headers['Authorization'] = f'Bearer {access_token}'
    headers['Content-Type'] = 'application/json'

    # construct req body
    data = ''

    print(f'\nRequest URL: {request_url}\n\n')

    resp = requests.get(request_url, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)

    # Create DB Object for Entry
    object_dict = dict(row)

    # object_list.append(object_dict)
    print(object_dict)

    # Explode and Shape Software Dict List Elements
    # for software in c_dict['software']:
    #     software_entry = explode_software(software)
    #     for k,v in software_entry.items():
    #         software_name = software_standard_filter(k)
    #         if software_name:
    #             object_dict[software_name] = v

    print('*'*50)

    return object_dict

In [ ]:
df_hardware = pd.DataFrame()

devices_hardware_list = []

for index, row in df_devices.iterrows():

    devices_hardware_list.append(hardware_api_req(row))


df_hardware = pd.DataFrame(devices_hardware_list)

# Shape Software DataFrame

### Set Index to device UID

### FillNA

### Set Index to device UID

# Initial Metrics and CSV Creation

## All Fields